# Note and Interval Determination in Various Tunings</h1>

In some circumstances, it may be useful to compare known frequencies to expected notes in various twelve-tone tuning schemes, including custom tunings. 
This program 
1. prompts user for reference concert pitch A4 in Hz
2. prompts user for two frequencies in Hz
3. determines the interval between those frequencies in Cents
4. compares frequencies and interval to custom tuning table in semicolon-delimited CSV file. 
5. returns note names with deviation ±50 Cts. and interval in semitones ±50 Cts for each custom tuning, relative to concert pitch A4.
For demonstration purposes, a tuning table was used with tuning deviation data sourced from https://www.instrument-tuner.com/temperaments.html
                                                                        
Each of these functions may be easily adapted for other purposes. 

### Load basic libraries </h3>

In [ ]:
import csv
import numpy as np

### Load tuning table </h3>
Loads note names and tuning deviations from a semicolon-delimited CSV file. It is assumed that the first row contains the names of the notes and the first column the names of the tunings. 
This function is designed for 12-tone systems and will throw up a warning if the tuning table doesn't match the expected 12-tone format. 

In [ ]:
def load_tuning_table(csv_file):
    tunings = {}
    with open(csv_file, newline='', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=';')
        headers = next(reader)[1:]  # Skip first column (tuning names), keep note names
        for row in reader:
            tuning_name = row[0]
            deviations = list(map(float, row[1:]))
            if len(deviations) == 12:
                tunings[tuning_name] = deviations
            else:
                print(f"Warning: {tuning_name} has {len(deviations)} deviations (expected 12). Skipping.")
    return headers, tunings

### Convert a frequency to the basis for MIDI note number, for reference concert pitch a4. </h3>
Adds 69 for A4 = Note 69. 
Deviations from the expected MIDI note frequency will result in decimals added to the note number. 

In [ ]:
def frequency_to_midi(frequency, a4):
    return 69 + 12 * np.log2(frequency / a4)

### Convert the MIDI note number into a note name with octave. </h3>

In [ ]:
def midi_to_note(midi_number, note_names):
    note_index = int(midi_number) % 12
    octave = (int(midi_number) // 12) - 1
    return f"{note_names[note_index]}{octave}"

### Determine closest note </h3>
Calls <code>frequency_to_midi</code> and <code>midi_to_note</code> functions to determine the closest MIDI note and its deviation, in cents, for a given frequency and tuning. 
Rounds the <code>midi_to_note</code> result for standardised MIDI note number and subtracts it from initial result to calculate deviation from standard equal temperament. Tuning deviation from tuning table is then applied. 
If total deviation exceeds ±50 cents, MIDI note is incremented to compensate. 

In [ ]:
def find_closest_note(frequency, note_names, tunings, a4):
    # Find the closest note in each tuning, ensuring deviations stay within ±50 cents.
    midi_number = frequency_to_midi(frequency, a4)
    closest_midi = round(midi_number)
    deviation_from_standard = (midi_number - closest_midi) * 100
    
    results = {}
    for tuning, deviations in tunings.items():
        note_index = closest_midi % 12
        tuning_deviation = deviations[note_index]
        total_deviation = deviation_from_standard - tuning_deviation

        # Ensure deviations stay within ±50 cents
        if total_deviation > 50:
            closest_midi += 1
            total_deviation -= 100
        elif total_deviation < -50:
            closest_midi -= 1
            total_deviation += 100
        
        note_name = midi_to_note(closest_midi, note_names)
        results[tuning] = (note_name, tuning_deviation, total_deviation)
    
    return results

### Calculate interval </h3>
Calculate the interval between the two frequencies for custom tuning. 
Interval is calculated in equal temperament before tuning deviations are applied. 
Rounds the interval result for semitones and subtracts it from initial result to calculate remaining cents. 
If total deviation exceeds ±50 cents, semitone count is incremented/decremented to compensate.

In [ ]:
def calculate_interval(freq1, freq2, tunedev1, tunedev2):
    # Calculate interval between two frequencies in cents and semitones, for custom tuning.
    if freq2 >= freq1:
        equal_interval_cents = 1200 * np.log2(freq2 / freq1)
        adjusted_interval_cents = equal_interval_cents - tunedev1 + tunedev2 # adjust for tuning deviations
    else: 
        equal_interval_cents = 1200 * np.log2(freq1 / freq2)
        adjusted_interval_cents = equal_interval_cents - tunedev2 + tunedev1 # adjust for tuning deviations
    
    # break up interval into semitones and cents
    semitones = int(round(adjusted_interval_cents / 100))
    remaining_cents = adjusted_interval_cents - (semitones * 100)

    # Ensure remaining cents is within ±50 cents
    if remaining_cents > 50:
        semitones += 1
        remaining_cents -= 100
    elif remaining_cents < -50:
        semitones -= 1
        remaining_cents += 100

    return semitones, remaining_cents

### Input frequency </h3>
Prompt user input for a valid frequency.

In [ ]:
def get_frequency_input(prompt):
    while True:
        try:
            value = float(input(prompt))
            if value > 0:
                return value
            else:
                print("Frequency must be greater than 0.")
        except ValueError:
            print("Invalid input. Please enter a number.")

### Main program. </h3> 
Calls all of the above functions and formats results for output. 
Tuning table with custom tuning deviations is loaded. 
Then frequencies input by user (<code>a4</code>, <code>freq1</code>, <code>freq2</code>) are passed to <code>find_closest_note</code> and <code>find_closest_note</code> compares them to the loaded tuning table for assignment to a particular note. 
The tuning deviations are then fed into <code>calculate_interval</code> to determine the interval within each tuning, in semitones ±50 cents. 
Absolute value of remaining cents to avoid confusing double negatives in print output formatting. 

In [ ]:
def main():
    csv_file = 'example_temperaments.csv' 
    # example data sourced from http://www.instrument-tuner.com/temperaments.html on 2024/08/26
    
    note_names, tunings = load_tuning_table(csv_file)


    a4 = get_frequency_input("Enter concert pitch in Hz: ")
    freq1 = float(input("Enter the first frequency in Hz: "))
    freq2 = float(input("Enter the second frequency in Hz: "))
    print()
    
    print(f"Results for each tuning, at concert pitch A4 = {a4}: ")
    results1 = find_closest_note(freq1, note_names, tunings, a4)
    results2 = find_closest_note(freq2, note_names, tunings, a4)

     
    for tuning in tunings:
        note1, tunedev1, dev1 = results1[tuning]
        note2, tunedev2, dev2 = results2[tuning]
    
        semitones, remaining_cents = calculate_interval(freq1, freq2, tunedev1, tunedev2)

        remaining_cents_abs = abs(remaining_cents) # Ensure returned cent interval is positive
        print(f"{tuning}: \n{note1} ({dev1:+.2f} cents) to {note2} ({dev2:+.2f} cents), \nInterval: {semitones} semitones {remaining_cents_abs:.2f} cents \n")

if __name__ == "__main__":
    main()